## Higher-Order Functions

A function that takes one or more functions as inputs or returns a function is a **higher-order function**.

Examples:

* `map(f, xs)` applies `f` to each element
* `filter(f, xs)`
* Function composition:

  ```python
  def compose1(f, g):
      def h(x):
          return f(g(x))
      return h
  ```

  Here `compose1` returns a new function `h` that composes `f` after `g`.

* Closures: returned functions “close over” their defining environment, remembering bindings like `f` and `g`.

Higher-order functions allow you to build general patterns and abstractions (e.g. decorators, pipelines).

## Functions as Arguments

These three functions clearly share a common underlying pattern.

In [ ]:
def sum_naturals(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k, k + 1
    return total

In [ ]:
def sum_cubes(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + k * k * k, k + 1
    return total

In [ ]:
def pi_sum(n):
    total, k = 0, 1
    while k <= n:
        total, k = total + 8 / ((4 * k - 3) * (4 * k - 1)), k + 1
    return total

The presence of such a common pattern is strong evidence that there is a useful abstraction waiting to be brought to the surface. Each of these functions is a summation of terms. As program designers, we would like our language to be powerful enough so that we can write a function that expresses the concept of summation itself rather than only functions that compute particular sums. We can do so readily in Python by taking the common template shown above and transforming the "slots" into formal parameters:

In [ ]:
def summation(n, term):
    total, k = 0, 1
    while k <= n:
        total, k = total + term(k), k + 1
    return total


def identity(x):
    return x


def sum_naturals(n):
    return summation(n, identity)


sum_naturals(10)

In [ ]:
def square(x):
    return x * x


summation(10, square)

In [ ]:
def pi_term(x):
    return 8 / ((4 * x - 3) * (4 * x - 1))


def pi_sum(n):
    return summation(n, pi_term)


pi_sum(1e6)

## Nested Definitions

The above examples demonstrate how the ability to pass functions as arguments significantly enhances the expressive power of our programming language. Each general concept or equation maps onto its own short function. One negative consequence of this approach is that the global frame becomes cluttered with names of small functions, which must all be unique. Another problem is that we are constrained by particular function signatures: the update argument to improve must take exactly one argument. Nested function definitions address both of these problems, but require us to enrich our environment model.

In [4]:
def average(x, y):
    return (x + y) / 2


def sqrt_update(x, a):
    return average(x, a / x)


def sqrt_close(x, a):
    return approx_eq(x * x, a)


def approx_eq(x, y, tolerance=1e-15):
    return abs(x - y) < tolerance


def improve(update, close, guess=1):
    while not close(guess):
        guess = update(guess)
    return guess


def sqrt(a):
    return improve(sqrt_update, sqrt_close)


def sqrt(a):
    def sqrt_update(x):
        return average(x, a / x)

    def sqrt_close(x):
        return approx_eq(x * x, a)

    return improve(sqrt_update, sqrt_close)

## Functions as Returned Values

We can achieve even more expressive power in our programs by creating functions whose returned values are themselves functions.

In [ ]:
def square(x):
    return x * x


def successor(x):
    return x + 1


def compose1(f, g):
    def h(x):
        return f(g(x))

    return h


# def f(x):
#     """Never called."""
#     return -x


square_successor = compose1(square, successor)
result = square_successor(12)

##  Example: Newton's Method

Newton's method is a classic iterative approach to finding the arguments of a mathematical function that yield a return value of 0. These values are called the zeros of the function. Finding a zero of a function is often equivalent to solving some other problem of interest, such as computing a square root.

In [2]:
def newton_update(f, df):
    def update(x):
        return x - f(x) / df(x)

    return update


def find_zero(f, df):
    def near_zero(x):
        return approx_eq(f(x), 0)

    return improve(newton_update(f, df), near_zero)


def square_root_newton(a):
    def f(x):
        return x * x - a

    def df(x):
        return 2 * x

    return find_zero(f, df)

In [5]:
square_root_newton(64)

8.0

## Currying

We can use higher-order functions to convert a function that takes multiple arguments into a chain of functions that each take a single argument.

In [6]:
def curried_pow(x):
    def h(y):
        return pow(x, y)

    return h


curried_pow(2)(3)

8

In [7]:
def map_to_range(start, end, f):
    while start < end:
        print(f(start))
        start = start + 1


map_to_range(0, 10, curried_pow(2))

1
2
4
8
16
32
64
128
256
512


## Lambda Expressions

So far, each time we have wanted to define a new function, we needed to give it a name. But for other types of expressions, we don't need to associate intermediate values with a name.

In [ ]:
def compose1(f, g):
    return lambda x: f(g(x))

In [8]:
s = lambda x: x * x

s(12)

144

## Decorators

Python provides special syntax to apply higher-order functions as part of executing a def statement, called a decorator. Perhaps the most common example is a trace.

In [1]:
def trace(fn):
    def wrapped(x):
        print("-> ", fn, "(", x, ")")
        return fn(x)

    return wrapped


@trace
def triple(x):
    return 3 * x


triple(12)

->  <function triple at 0x10a8c28e0> ( 12 )


36

## Recursive Functions

When a function calls itself (directly or indirectly), that’s recursion.

In [ ]:
def sum_digits(n):
    """Return the sum of the digits of positive integer n."""
    if n < 10:
        return n
    else:
        all_but_last, last = n // 10, n % 10
        return sum_digits(all_but_last) + last


### Anatomy of Recursion

Key parts:

1. **Base case** — the stopping condition
2. **Recursive case** — the function reduces the problem and calls itself

Example (factorial):

```python
def fact(n):
    if n == 0:
        return 1
    else:
        return n * fact(n - 1)
```


### Mutual Recursion

Two (or more) functions call each other. For example:

In [ ]:
def is_even(n):
    if n == 0:
        return True
    else:
        return is_odd(n - 1)


def is_odd(n):
    if n == 0:
        return False
    else:
        return is_even(n - 1)


result = is_even(4)

In [ ]:
def is_even(n):
    if n == 0:
        return True
    else:
        if (n - 1) == 0:
            return False
        else:
            return is_even((n - 1) - 1)


### Tree Recursion

Some recursive algorithms branch (call themselves multiple times). For example, naive Fibonacci:

In [ ]:
def fib(n):
    if n == 1:
        return 0
    if n == 2:
        return 1
    else:
        return fib(n - 2) + fib(n - 1)


result = fib(6)


This is *tree recursion*: the calls form a branching tree. Its cost is exponential.